In [144]:
import glob
import os
import numpy as np
import pandas as pd
from datetime import datetime
csv_path = '../price_predict/vegetable_price/original_vegetable_csv/'
vegetable_names = ['청양', '시금치(일반)', '토마토(일반)', '파프리카(일반)', '깻잎(일반)', '미나리(일반)', '양파(일반)', '풋고추(전체)']
csv_files = glob.glob(csv_path + '*.csv')

In [145]:
class VegetablePriceReader:
    # csv_file를 받아서 초기화하는 생성자 함수
    def __init__(self, csv_file):
        self.vege_name = os.path.basename(csv_file.split()[0])
        self.df = pd.read_csv(csv_file)
        self.df = self.df.drop(self.df.columns[1], axis=1).fillna(0)


    # '일자' 열을 확인하고 오늘 날짜 이전의 모든 행을 필터링하는 함수
    def filter_rows(self):
        self.df['일자'] = pd.to_datetime(self.df['일자'].astype(str), format='%Y%m%d')
        today = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
        self.df = self.df[self.df['일자'] <= today]


    # 위의 네 가지 함수를 순서대로 실행하는 함수
    def process(self):
        col_name = self.df.columns[1]
        self.df = self.df.rename(columns={col_name: self.vege_name + " " + col_name})
        self.filter_rows()

        return self.df.iloc[:,1]



In [146]:

# 오늘 날짜를 구합니다.
today = datetime.today().strftime('%Y-%m-%d')

# 날짜 범위를 생성합니다.
date_range = pd.date_range(start='2023-01-01', end=today)

# 날짜 범위를 데이터프레임에 추가합니다.
df = pd.DataFrame(date_range, columns=['일자'])
df['일자'] = df['일자'].dt.strftime('%Y-%m-%d')

for csv_path in csv_files:
    reader = VegetablePriceReader(csv_path)
    df = pd.concat([df, reader.process()], axis=1)



In [147]:
df.to_json('output.json', orient='records')

In [148]:
with open('output.json', 'r') as f:
    data = json.load(f)


In [151]:
reader.df

,일자,풋고추(전체) 평균가격(원/kg)
0,2023-01-01,0.0
1,2023-01-02,6600.0
2,2023-01-03,6420.0
3,2023-01-04,6430.0
4,2023-01-05,6340.0
...,...,...
139,2023-05-23,7020.0
140,2023-05-24,6990.0
141,2023-05-25,6960.0
142,2023-05-26,6930.0
